# Posting an Image to BlueSky
This Notebook will give an example of how to use Python to post an image with descriptive text to BlueSky. It will show how to create a session where we log in to our BlueSky account, pull an image from somewhere, automatically write the post text and then publish it. 

As you will see, this is actually incredibly simple and only requires brand new Python package to achieve.

## Required Imports
First, we need some very basic Python packages to create the post. Which base ones you need will depend on the kind of bot you want to create. Here, we pull the image and description of it from two different sources in the Galaxy Zoo archives. Therefore, we need to interact with urls and .csvs hidden away on different servers. This is possible with a combination of the requests, pandas and pillow Python packages. 

In [3]:
import pandas as pd
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import os

I also import BytesIO here, as it allows us to read the requested image directly into the memory of the workflow, rather than having to save it somewhere. It also allows us to post the image very easily.

However, the one new Python package we need is the atproto python package. For a full description, please go to the docs [here](https://atproto.blue/en/latest/). This package is still very much under construction, but it allows us to easily and efficiently interface with the BlueSky API. Here, we will only use it to make posts, but this package can also be used to like, repost and comment on existing posts. From this package, we import the Client class.

In [2]:
from atproto import Client

## Using atproto to Sign In
To make a post, we must first sign in to our (or our bots) BlueSky account. For this, we will require only our BlueSky username and password. I highly recommend that you DO NOT write these details in your code. For this example when testing your bot, I'd recommend that you make your details environment variables and get them using the os Python package.

Note, when we write the GitHub workflow in a later Notebook, we will load in our BlueSky details using Secret Access Tokens. These are set up on GitHub itself, and are fully encrypted and hidden from any other users. However, from a programming perspective, they act precisely like environment variables. Thus, when we write the script and workflow of the bot, the below code will still work.

In [5]:
client = Client() # Initialise the Client class.

In [36]:
usrname = os.getenv('BLUESKY_USRNAME')

In [ ]:
pwd = os.getenv('BLUESKY_PWD')

In [38]:
_ = client.login(usrname, pwd)

While I have subdued the output of the function here, you will either get a long string explaining that you are signed in or a long string saying that you aren't. If you have failed to sign in, there will be a piece of string in the output dictionary which will tell you why. Often, it is an 'Access Denied' error which means there is something wrong with the syntax, or text in your password or username. While there are many things that can go wrong, the two that tripped me up the most were:

    1. Some characters get converted to unicode when read in (though, this shouldn't happen is you use the environment variable method). So, your password or username may be corrupted between here and signing in.
    
    2. There are protected environment variables that cannot be overwritten. If you try to set one of these they will actually retain their original value. Therefore, your login credentials will be incorrect. Test with different environment variable names if necessary, or look up what the protected ones are.

## Getting the Galaxy Image
Now that we are logged in, we are ready to start building our BlueSky post! This example bot, of course, publishes galaxy images with some descriptive post text. So, let's do this piece by piece. 

First, we want to get the galaxy image. As stated previously, this image is pulled from a url which is contained in a master .csv file hosted on GitHub. We read in the .csv file from it's URL on GitHub (you must use the RAW link, and not the DIRECT link). This has also been stored as an environment variable.

In [ ]:
cat_path = os.getenv('CATALOGUE_PATH')
full_csv = pd.read_csv(cat_path)

In our case, this .csv contains of the order of millions of galaxies across all the Galaxy Zoo Projects. As we want to post one, and our sample is not so small to worry about duplication of rows yet, we simply use the pandas sample function to randomnly select a row in the .csv and extract the URL of the image.

In [ ]:
gal_row = pd.read_csv(cat_path).sample(1)
url = gal_row['image_url'].iloc[0]

With the URL, we can now use the requests package to pull the image data and convert this to Bytes with the BytesIO package.

In [ ]:
def pull_galaxy_image(url):
    response = requests.get(url)
    img_data = BytesIO(response.content)
    return img_data 

In [ ]:
img_data = pull_galaxy_image(url)

This small function that we have written returns the galaxy image in a format that can be directly uploaded to BlueSky. Therefore, we do nothing further with it here. If you wish to manipulate and look at the image yourself, you can use the Pillow package to convert it from a BytesIO object to an image.

In [ ]:
image = Image.open(img_data)
image

We now have our image for our post! Of course, how you pull the image will depend on how you store them. This is just an example of how we have done this.

Now that we have an image to post, we need to automatically write the post information!

## Writing the Post
This will heavily depend on what you want your bot to actually do. With our bot, we have a set of informative columns in our DataFrame row which have everything we need to write a brief description of our galaxy. We read the row we have selected previously and extract the different pieces of information and format them into a post.

The example function below can act as inspiration for what you would like to do!

In [118]:
def create_post_text(row):
    if type(row.redshift.iloc[0]) == str:
        z = row.redshift.iloc[0]
    else:
        z = "%.2f" % ( row.redshift.iloc[0] )
    ra = "%.5f" % row.ra.iloc[0]
    dec = "%.5f" % row.dec.iloc[0]
    clsf = row.galaxy_description.iloc[0]
    survey = row.imaging.iloc[0]
    if "CANDELS-COODS" in survey:
        survey = 'CANDELS-GOODS'
       
    project = row.project.iloc[0]
    if 'Hubble' in project:
        instr = 'Hubble Space Telescope'
    elif 'CANDELS' in project:
        instr = 'Hubble Space Telescope'
    elif 'Galaxy Zoo 2' in project:
        instr = 'Apache Point 2.5m Telescope'

    t_lookback = row.t_lookback.iloc[0]
    if t_lookback < 1:
        tmp = t_lookback * 1000
        t_lookback_string = '%.3f million years' % tmp
    else:
        t_lookback_string = '%.2f billion years' % t_lookback

    random_no = np.random.random()

    if random_no < (1/24):
        
        metadata = (
    """A {}, observed with the {} in the {} survey.
    
It is at redshift {} (lookback time {}) with coordinates ({}, {}).

This classification was made in the {} project.
\U0001f52d
    """.format(
                clsf, instr, survey, z, t_lookback_string, ra, dec, project
            )
        )
    else:
        metadata = (
    """A {}, observed with the {} in the {} survey.
    
It is at redshift {} (lookback time {}) with coordinates ({}, {}).

This classification was made in the {} project.
    """).format(
                clsf, instr, survey, z, t_lookback_string, ra, dec, project
            )

    return metadata

In [ ]:
post_string = create_metadata(gal_row)

Upon running the above function, we can look at the post string we have created.

In [50]:
print(post_string)

A spiral galaxy, observed with the Hubble Space Telescope in the COSMOS survey.
    
It is at redshift 0.51 (lookback time 5.28 billion years) with coordinates (150.52525, 2.82778).

This classification was made in the GZ: Hubble project.
🔭
    


We have also included a chance that a telescope emoji will be added to our post string. This is to cross post our publication with the Astronomy feed. You can look up the unicode for the emoji that conducts the cross post you want, and follow suit if you would like! Note that to work, your bot must also be allowed to post on those feeds. So, talk to the respective administrators if needed.

## Publishing the Post
So, we have now got the two main things we need for our post: the image and the string. We just have to post it now and add some alternative text. This can be done with a single function within the client class we initialised and logged in with earlier.

We pass this, the post string and image (in Bytes) to our own post function. We also send some other parameters to build some dynamic alternative text as well.

In [ ]:
def post(image, post_string, client, clsf, project):

    alt_im_text = 'A {} from the {} project.'.format(clsf, project)

    response = (
        client.send_image(
            text = post_string, 
            image = image, 
            image_alt = alt_im_text
        )
    )
    
    return response

In [53]:
# Posting
response = post(image, post_string, client, gal_row.galaxy_description.iloc[0], gal_row.project.iloc[0])
print(response)

uri='at://did:plc:6itg6aj2yv3gug4ah7ejicsu/app.bsky.feed.post/3kr7bdrkdzf24' cid='bafyreidtdbaowxdriuqnkecz3424gu2mm7fpq6gg3hwg37736wia2ejyli'


Running our function leads to our posting! The response shows us the ID and uri of the post. However, if you now go to your BlueSky account, you will be able to see the post yourself!

So, we now have the outline of the bot that will be making these posts. We just have to automate this... This is shown in the Python script gz-bot.py where, essentially, the above is done just in a way the GitHub Actions can run them. The main challenge is now getting the workflow running with regular triggers.